In [ ]:
!pip install fosforml
!pip install statsmodels

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [2]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [3]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [4]:
sf_df = my_session.sql("select * from {}".format(df))

In [5]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [6]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                      0
TENURE_MONTHS                    0
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
CITY                             0
DISTANCE                         0
DEGREE_CLEAN                     0
ETHNICITY                        0
MARITAL_STATUS                   0
ROLE                             0
COMPANY_NAME                     0
ORGANIZATION_TYPE                0
ORGANIZATION_OWNERSHIP           0
STATE                            0
COUNTRY                          0
GENDER                           0
WORK_LIFE_BALANCE                0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFA

In [7]:
Original_df = pandas_df.dropna()

In [8]:
print(Original_df.isnull().sum())

EMPLOYEE_ID                  0
TENURE_MONTHS                0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
CITY                         0
DISTANCE                     0
DEGREE_CLEAN                 0
ETHNICITY                    0
MARITAL_STATUS               0
ROLE                         0
COMPANY_NAME                 0
ORGANIZATION_TYPE            0
ORGANIZATION_OWNERSHIP       0
STATE                        0
COUNTRY                      0
GENDER                       0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
dtype: int64


In [9]:
# Ensure date columns are parsed correctly
date_columns = ['SCHOOL_ENDDATE', 'JOB_STARTDATE', 'JOB_ENDDATE']
for col in date_columns:
    Original_df[col] = pd.to_datetime(Original_df[col], errors='coerce')

In [10]:
# Display the cleaned DataFrame
print(Original_df.head())

  EMPLOYEE_ID  TENURE_MONTHS  BIRTH_YEAR  AGE  SENIORITY SCHOOL_ENDDATE  \
0    94215814             38        1981   43          1     2006-01-01   
1    20227353             60        1991   33          1     2009-01-01   
3    15048356              9        1992   32          1     2014-01-31   
4    91151756             38        1981   43          1     2006-01-01   
6    91039761              5        1987   37          1     2014-01-31   

  JOB_STARTDATE JOB_ENDDATE       CITY   DISTANCE  ... WORK_LIFE_BALANCE  \
0    2016-06-01  2019-08-01  Charlotte   <2 miles  ...              Best   
1    2016-06-01  2021-06-01  Charlotte   <2 miles  ...            Better   
3    2016-10-01  2017-07-01  Charlotte   <2 miles  ...               Bad   
4    2016-06-01  2019-08-01  Charlotte   <2 miles  ...            Better   
6    2018-02-01  2018-07-01     Dallas  >10 miles  ...              Best   

  BUSINESS_TRAVEL ENVIRONMENT_SATISFACTION    SALARY JOB_SATISFACTION  \
0   Travel Rarely  

In [16]:
# Convert columns to numeric, forcing errors to NaN
Original_df['TENURE_MONTHS'] = pd.to_numeric(Original_df['TENURE_MONTHS'], errors='coerce')
Original_df['AGE'] = pd.to_numeric(Original_df['AGE'], errors='coerce')
Original_df['SENIORITY'] = pd.to_numeric(Original_df['SENIORITY'], errors='coerce')


In [24]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encode ORGANIZATION_TYPE and DEGREE_CLEAN
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = encoder.fit_transform(Original_df[['ORGANIZATION_TYPE']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['ORGANIZATION_TYPE']))

# Combine encoded features with the original DataFrame
Original_df = pd.concat([Original_df.reset_index(drop=True), encoded_df], axis=1)


In [25]:
Original_df = Original_df.dropna()

In [26]:
import statsmodels.api as sm

# Assuming 'CHURN' is your target variable
y = Original_df['CHURN']
X = Original_df[encoder.get_feature_names_out(['ORGANIZATION_TYPE'])]

In [27]:
# Add a constant to the features
X = sm.add_constant(X)

In [28]:
# Split data into training and testing sets
train_size = int(len(y) * 0.8)
y_train, y_test = y[:train_size], y[train_size:]
X_train, X_test = X[:train_size], X[train_size:]

In [34]:
# Fit ARIMAX model with increased iterations and initial parameters
model = sm.tsa.ARIMA(y_train, exog=X_train, order=(1, 1, 1))
model_fit = model.fit(maxiter=500, start_params=[0.5, 0.5, 0.5, 0.5, 0.5])  # Adjust the number of initial parameters as needed

TypeError: fit() got an unexpected keyword argument 'maxiter'

In [23]:
# Make predictions for the next 12 months
future_steps = 12
future_exog = X_test.iloc[-1:].append([X_test.iloc[-1:]] * (future_steps - 1), ignore_index=True)  # Adjust exogenous variables as needed
predictions = model_fit.forecast(steps=future_steps, exog=future_exog)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Make predictions
predictions = model_fit.forecast(steps=len(y_test), exog=X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
# Make predictions for the next 12 months
future_steps = 12
future_exog = X_test.iloc[-1:].append([X_test.iloc[-1:]] * (future_steps - 1), ignore_index=True)  # Adjust exogenous variables as needed
predictions = model_fit.forecast(steps=future_steps, exog=future_exog)

print(predictions)

In [ ]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [ ]:
df

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])

In [ ]:
df.set_index('JOB_ENDDATE', inplace = True)

In [ ]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]

In [ ]:
# Group by categorical columns and resample monthly, then sum CHURN
New_df = df.groupby(["DISTANCE", "DEGREE_CLEAN", "ROLE", "COMPANY_NAME", "ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP"]).resample('M').agg({'CHURN': 'sum'}).reset_index()

In [ ]:
# Filter rows where CHURN is greater than 0
df1 = New_df[New_df['CHURN'] > 0]

In [ ]:
df1

In [ ]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [ ]:
# Ensure the index is unique
df1 = df1[~df1.index.duplicated(keep='first')]

In [ ]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

In [ ]:
# Ensure date column is in datetime format
df1['JOB_ENDDATE'] = pd.to_datetime(df1['JOB_ENDDATE'])

In [ ]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]


In [ ]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [ ]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

In [ ]:
# Split data into training and testing sets
train = df1[:'2024-08']
test = df1['2024-09':]

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [ ]:
monthly_data